# 06 — Multi-Agent Orchestrator

Ein Supervisor-Agent, der freie Benutzeranfragen entgegennimmt und Spezialagenten koordiniert:
- **Compliance Classifier** → Risikostufen-Klassifizierung
- **Obligation Advisor** → Compliance-Checkliste
- **Regulation Q&A** → offene Fragen

Demonstriert das **"Agents as Tools"**-Muster — jeder Spezialist wird als aufrufbares Tool für den Orchestrator gewrappt.

Beispiel: *"Ich baue ein Gesichtserkennungssystem für die Flughafensicherheit, wir sind der Anbieter. Was muss ich wissen?"* → Orchestrator ruft Classifier auf (Hochrisiko), leitet an Obligation Advisor weiter (Anbieter-Checkliste), nutzt Q&A für weitere Fragen, und erstellt einen zusammenfassenden Bericht.

In [1]:
import sys
sys.path.insert(0, "..")

import json
from src.tools import init_tools, TOOL_SCHEMAS, TOOL_DISPATCH
from src.agent import run_agent
from src.embeddings import get_openai_client

LANG = "de"  # change to "en" for English

client = get_openai_client()
init_tools(chroma_path="../chroma_db", lang=LANG)

## Spezialagenten als Tools definieren

In [2]:
# System-Prompts für Spezialagenten
CLASSIFIER_PROMPT = """Du bist ein Klassifikator für die EU-KI-Verordnung. Bestimme die Risikostufe des KI-Systems.
Prüfe Art. 5 (verboten), Art. 6 + Anhang III (Hochrisiko), Art. 50 (Transparenz/begrenzt).
Nutze die Tools für den Verordnungstext. Antworte mit knappem JSON: risk_level, confidence, reasoning, 
prohibited_check, high_risk_check, transparency_check und citations. Antworte auf Deutsch."""

ADVISOR_PROMPT = """Du bist ein Pflichten-Berater für die EU-KI-Verordnung. Erstelle eine Compliance-Checkliste für die gegebene Rolle und Risikostufe.
Nutze die Tools für die tatsächlichen Pflichten. Antworte mit knappem JSON: role, risk_level, obligations 
(je mit obligation, article, description, priority), summary und citations. Antworte auf Deutsch."""

QA_PROMPT = """Du bist ein Experte für die EU-KI-Verordnung. Beantworte die Frage mithilfe der Tools.
Verwende Inline-Zitate [Art. X]. Antworte mit knappem JSON: answer, citations, confidence. Antworte auf Deutsch."""


def classify_ai_system(system_description: str) -> str:
    """Run the classifier agent on a system description."""
    response, _ = run_agent(
        client=client,
        system_prompt=CLASSIFIER_PROMPT,
        user_message=f"Klassifiziere dieses KI-System:\n\n{system_description}",
        tools=TOOL_SCHEMAS,
    )
    return response


def get_obligations(role: str, risk_level: str, system_context: str = "") -> str:
    """Run the obligation advisor agent."""
    msg = f"Rolle: {role}\nRisikostufe: {risk_level}"
    if system_context:
        msg += f"\nKontext: {system_context}"
    response, _ = run_agent(
        client=client,
        system_prompt=ADVISOR_PROMPT,
        user_message=msg,
        tools=TOOL_SCHEMAS,
    )
    return response


def ask_regulation(question: str) -> str:
    """Run the Q&A agent on a question."""
    response, _ = run_agent(
        client=client,
        system_prompt=QA_PROMPT,
        user_message=question,
        tools=TOOL_SCHEMAS,
    )
    return response

In [3]:
# Orchestrator tool schemas — each specialist agent is a tool
ORCHESTRATOR_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "classify_ai_system",
            "description": "Classify an AI system's risk level under the EU AI Act. Returns risk classification with citations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "system_description": {"type": "string", "description": "Description of the AI system to classify"},
                },
                "required": ["system_description"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_obligations",
            "description": "Get compliance obligations for a given role and risk level. Returns a checklist with article citations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "role": {"type": "string", "enum": ["provider", "deployer", "importer", "distributor"]},
                    "risk_level": {"type": "string", "enum": ["unacceptable", "high", "limited", "minimal"]},
                    "system_context": {"type": "string", "description": "Additional context about the AI system"},
                },
                "required": ["role", "risk_level"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "ask_regulation",
            "description": "Ask any question about the EU AI Act. Returns an answer with article citations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string", "description": "Question about the EU AI Act"},
                },
                "required": ["question"],
            },
        },
    },
]

# Dispatch for orchestrator tools
ORCHESTRATOR_DISPATCH = {
    "classify_ai_system": lambda args: classify_ai_system(**args),
    "get_obligations": lambda args: get_obligations(**args),
    "ask_regulation": lambda args: ask_regulation(**args),
}

In [4]:
ORCHESTRATOR_SYSTEM_PROMPT = """Du bist ein EU-KI-Verordnung Compliance-Orchestrator. Du koordinierst Spezialagenten für umfassende regulatorische Beratung.

Verfügbare Spezialagenten (als Tools nutzen):
1. classify_ai_system — Risikostufe eines KI-Systems klassifizieren
2. get_obligations — Compliance-Checkliste für Rolle + Risikostufe erstellen
3. ask_regulation — Beliebige Frage zur EU-KI-Verordnung beantworten

Workflow für eine umfassende Analyse:
1. Zuerst: Risikostufe des KI-Systems mit classify_ai_system bestimmen
2. Dann: Pflichten für die Rolle des Nutzers mit get_obligations abrufen (Ergebnis der Klassifizierung nutzen)
3. ask_regulation für zusätzliche spezifische Fragen nutzen
4. Alle Ergebnisse in einer klaren Zusammenfassung synthetisieren

Deine finale Antwort soll ein gut strukturierter Bericht sein, der alle Spezialisten-Ergebnisse kombiniert.
Antworte auf Deutsch."""

## Vollständige Orchestrierung: Gesichtserkennung am Flughafen

In [5]:
%%time
from datetime import datetime

AGENT_LABELS = {
    "classify_ai_system": "Compliance Classifier",
    "get_obligations": "Obligation Advisor",
    "ask_regulation": "Regulation Q&A",
}

def live_tracker(event, data):
    label = AGENT_LABELS.get(data["name"], data["name"])
    ts = datetime.fromtimestamp(data["timestamp"]).strftime("%H:%M:%S")
    if event == "tool_call_start":
        args_summary = ", ".join(f"{k}={repr(v)[:60]}" for k, v in data["arguments"].items())
        print(f"[{ts}] → {label} gestartet ({args_summary})")
    elif event == "tool_call_end":
        print(f"[{ts}] ✓ {label} fertig ({data['duration']:.1f}s)")

response, messages = run_agent(
    client=client,
    system_prompt=ORCHESTRATOR_SYSTEM_PROMPT,
    user_message="""Ich baue ein Gesichtserkennungssystem für die Sicherheitskontrolle am Flughafen.
Wir sind der Anbieter dieses Systems. Das System verifiziert die Identität von Passagieren, 
indem es ihr Gesicht mit dem Passfoto am Boarding-Gate abgleicht.

Was muss ich bezüglich der EU-KI-Verordnung beachten?""",
    tools=ORCHESTRATOR_TOOLS,
    tool_dispatch=ORCHESTRATOR_DISPATCH,
    max_turns=15,
    on_tool_call=live_tracker,
)

print(f"\n{'='*80}")
print("ORCHESTRATOR-BERICHT")
print('='*80)
print(response)

[12:19:57] → Compliance Classifier gestartet (system_description='Gesichtserkennungssystem zur Sicherheitskontrolle am Flugha)
[12:20:04] ✓ Compliance Classifier fertig (6.5s)
[12:20:05] → Obligation Advisor gestartet (role='provider', risk_level='high', system_context='Gesichtserkennungssystem zur Sicherheitskontrolle am Flugha)
[12:20:28] ✓ Obligation Advisor fertig (22.9s)
[12:20:30] → Regulation Q&A gestartet (question='Welche Transparenzpflichten gelten für Anbieter von Hochris)
[12:20:36] ✓ Regulation Q&A fertig (6.0s)

ORCHESTRATOR-BERICHT
Bericht zur EU-KI-Verordnung für Ihr Gesichtserkennungssystem zur Sicherheitskontrolle am Flughafen

1. Klassifizierung des KI-Systems:
Ihr Gesichtserkennungssystem, das die Identität von Passagieren am Boarding-Gate durch Abgleich mit Passfotos verifiziert, wird nach der EU-KI-Verordnung als Hochrisiko-KI-System eingestuft. Dies ergibt sich aus Anhang III, Kapitel 1, da biometrische Verifizierungssysteme, besonders in sicherheitskritischen Be

## Orchestrierung: KI-gestützte medizinische Diagnose

In [ ]:
%%time

response2, messages2 = run_agent(
    client=client,
    system_prompt=ORCHESTRATOR_SYSTEM_PROMPT,
    user_message="""Wir setzen in unserem Krankenhaus ein KI-Diagnosetool ein, das medizinische Bilder 
(Röntgen, MRT) analysiert, um potenzielle Krebserkrankungen zu erkennen. Wir betreiben ein 
Drittsystem, wir entwickeln es nicht selbst.

Welche Pflichten haben wir? Gibt es spezifische Anforderungen für medizinische KI?""",
    tools=ORCHESTRATOR_TOOLS,
    tool_dispatch=ORCHESTRATOR_DISPATCH,
    max_turns=15,
    on_tool_call=live_tracker,
)

print(f"\n{'='*80}")
print("ORCHESTRATOR-BERICHT")
print('='*80)
print(response2)

# Orchestrierung: Nur generelle Q&A Frage zum AI Act -> kein Bericht soll erstellt werden

In [7]:
%%time

response3, messages3 = run_agent(
    client=client,
    system_prompt=ORCHESTRATOR_SYSTEM_PROMPT,
    user_message="""Kannst du mir die wichtigsten Punkte des EU AI Acts in 5 Sätzen zusammenfassen?""",
    tools=ORCHESTRATOR_TOOLS,
    tool_dispatch=ORCHESTRATOR_DISPATCH,
    max_turns=15,
    on_tool_call=live_tracker,
)

print(f"\n{'='*80}")
print("ORCHESTRATOR-BERICHT")
print('='*80)
print(response3)

[12:26:34] → Regulation Q&A gestartet (question='Fasse die wichtigsten Punkte des EU AI Act in 5 Sätzen zusa)
[12:26:39] ✓ Regulation Q&A fertig (5.2s)

ORCHESTRATOR-BERICHT
Der EU AI Act stellt sicher, dass KI-Systeme in der EU unter Berücksichtigung von Gesundheit, Sicherheit und Grundrechten verwendet werden, mit strengen Vorschriften für Hochrisiko-KI. Anbieter von allgemeinen KI-Modellen müssen transparente Informationen und technische Dokumentationen zum Training und Gebrauch bereitstellen. Hochrisiko-KI-Systeme sind zu registrieren und müssen Maßnahmen für menschliche Aufsicht und Kontrolle enthalten. Die Verordnung legt großen Wert auf Transparenz, Nachvollziehbarkeit, Antidiskriminierung und den Schutz der Rechte von Urhebern und Betroffenen. Ziel ist eine harmonisierte Regelung in der EU, die ethische Leitlinien und Menschenrechte respektiert (vgl. Erwägungsgründe 7, 107; Anhänge IV, VIII, XI, XII).
CPU times: user 39.6 ms, sys: 10.5 ms, total: 50.1 ms
Wall time: 9.15 s


# Orchestrierung: Nur frage nach allgemeiner Checkliste

In [8]:
%%time

response4, messages4 = run_agent(
    client=client,
    system_prompt=ORCHESTRATOR_SYSTEM_PROMPT,
    user_message="""Kannst du mir eine allgemeine Compliance Checkliste erstellen?""",
    tools=ORCHESTRATOR_TOOLS,
    tool_dispatch=ORCHESTRATOR_DISPATCH,
    max_turns=15,
    on_tool_call=live_tracker,
)

print(f"\n{'='*80}")
print("ORCHESTRATOR-BERICHT")
print('='*80)
print(response4)


ORCHESTRATOR-BERICHT
Um eine präzise und hilfreiche allgemeine Compliance-Checkliste zur EU-KI-Verordnung zu erstellen, benötige ich einige Informationen:

1. Beschreibung des KI-Systems (z.B. Zweck, Funktionen, Anwendungsgebiet)
2. Deine Rolle im Bezug zum KI-System (z.B. Anbieter, Betreiber, Importeur, Händler)

Diese Angaben helfen mir, das KI-System richtig zu klassifizieren und passende Pflichten aufzulisten.

Bitte gib mir diese Details, damit ich loslegen kann.
CPU times: user 5.21 ms, sys: 2.14 ms, total: 7.35 ms
Wall time: 2.03 s
